In [1]:
import os
base_dir = './'
non_recyclable_dir = os.path.join(base_dir, 'non-recyclable')
recyclable_dir = os.path.join(base_dir, 'recyclable')
test_dir = os.path.join(base_dir, 'test')
test_nonrecyclable_dir = os.path.join(test_dir, 'test_nonrecyclable')
test_recyclable_dir = os.path.join(test_dir, 'test_recycleable')

In [2]:
print("Non-recyclable images:", len(os.listdir(non_recyclable_dir)))
print("Recyclable images:", len(os.listdir(recyclable_dir)))
print("Test non-recyclable images:", len(os.listdir(test_nonrecyclable_dir)))
print("Test recyclable images:", len(os.listdir(test_recyclable_dir)))

Non-recyclable images: 40
Recyclable images: 40
Test non-recyclable images: 10
Test recyclable images: 10


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 200, 200
batch_size = 10

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    classes=['recyclable', 'non-recyclable'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    base_dir,
    classes=['recyclable', 'non-recyclable'],
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

ModuleNotFoundError: No module named 'tensorflow'

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=8
)

Epoch 1/10
8/8 [==============================] - 3s 271ms/step - loss: 1.2559 - accuracy: 0.4500 - val_loss: 0.6745 - val_accuracy: 0.7125
Epoch 2/10
8/8 [==============================] - 2s 228ms/step - loss: 0.7066 - accuracy: 0.5500 - val_loss: 0.6538 - val_accuracy: 0.5000
Epoch 3/10
8/8 [==============================] - 2s 208ms/step - loss: 0.6484 - accuracy: 0.6375 - val_loss: 0.5769 - val_accuracy: 0.8125
Epoch 4/10
8/8 [==============================] - 2s 225ms/step - loss: 0.5489 - accuracy: 0.7375 - val_loss: 0.4613 - val_accuracy: 0.8375
Epoch 5/10
8/8 [==============================] - 2s 208ms/step - loss: 0.4048 - accuracy: 0.8375 - val_loss: 0.2525 - val_accuracy: 0.9375
Epoch 6/10
8/8 [==============================] - 2s 208ms/step - loss: 0.2604 - accuracy: 0.9000 - val_loss: 0.1579 - val_accuracy: 0.9500
Epoch 7/10
8/8 [==============================] - 2s 230ms/step - loss: 0.1623 - accuracy: 0.9375 - val_loss: 0.1594 - val_accuracy: 0.9375
Epoch 8/10
8/8 [====

In [13]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

2/2 [==============================] - 0s 109ms/step - loss: 3.4391 - accuracy: 0.3500
Test accuracy: 0.3499999940395355


In [17]:
import numpy as np
from tensorflow.keras.preprocessing import image

def load_and_predict(img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    img_tensor /= 255.

    prediction = model.predict(img_tensor)
    return "recyclable" if prediction[0] < 0.5 else "non-recyclable"

test_img_paths = [
    os.path.join(test_recyclable_dir, 'QLCZ2890.jpg'),
    os.path.join(test_nonrecyclable_dir, 'WRAQ6784.jpg') 
]

for path in test_img_paths:
    result = load_and_predict(path)
    print(f'{path}: {result}')


1/1 [==============================] - 0s 23ms/step
./test\test_recycleable\QLCZ2890.jpg: recyclable
1/1 [==============================] - 0s 30ms/step
./test\test_nonrecyclable\WRAQ6784.jpg: non-recyclable
